In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import keras_tuner as kt

from tensorflow import data
from tensorflow.keras import Sequential, Input, Model
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D, Softmax
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.metrics import BinaryAccuracy
from tensorflow.keras.applications import resnet50

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
import sys
sys.path.append("../")
from util import data_prep as prep
from util import models as m

In [3]:
root_data_path = f"D:/malaria_data/cell_images_split"
seed = 2049
# Resize using "area" method
# Multiples of 32 work best with ResNet
resnet_inp_shape = (128, 128, 3)
val_split = 0.2
batch_size = 32
n_epochs = 4
min_fine_tune_lr = 1e-5
last_layer_options = ["conv2_block3_out", "conv3_block4_out", "conv4_block6_out", "conv5_block3_out"]

In [4]:
# model_0 = resnet50.ResNet50(weights="imagenet", include_top=False, input_shape=resnet_inp_shape)
# model_0.summary()

# Functions

# Data

## Data split

In [5]:
train_datagen = ImageDataGenerator(
    horizontal_flip=True,
    vertical_flip=True,
    # Make sure to preprocess the same as the original model
    preprocessing_function=resnet50.preprocess_input,
    # Use 20% of data for validation
    validation_split=val_split
)

In [6]:
test_datagen = ImageDataGenerator(
    preprocessing_function=resnet50.preprocess_input
)

## Generators

In [7]:
train_generator = train_datagen.flow_from_directory(
    f"{root_data_path}/train",
    target_size=resnet_inp_shape[:2],
    batch_size=batch_size,
    # Want uninfected to be labeled "0"
    classes={"uninfected": 0, "parasitized": 1},
    class_mode="binary",
    seed=seed,
    subset="training"
)

val_generator = train_datagen.flow_from_directory(
    f"{root_data_path}/train",
    target_size=resnet_inp_shape[:2],
    batch_size=batch_size,
    classes={"uninfected": 0, "parasitized": 1},
    class_mode="binary",
    seed=seed,
    subset="validation"
)

Found 17638 images belonging to 2 classes.
Found 4408 images belonging to 2 classes.


In [8]:
test_generator = test_datagen.flow_from_directory(
    f"{root_data_path}/test",
    target_size=resnet_inp_shape[:2],
    batch_size=batch_size,
    classes={"uninfected": 0, "parasitized": 1},
    class_mode=None,
    shuffle=False
)

Found 5512 images belonging to 2 classes.


# Build model

In [9]:
# hp = kt.HyperParameters()
hypermodel = m.ResNet50TLHyperModel(
    inp_shape=resnet_inp_shape,
    name="resnet_malaria",
    last_layer_options=last_layer_options,
    min_fine_tune_lr=min_fine_tune_lr,
    max_epochs_1=n_epochs,
    max_epochs_2=n_epochs // 2
)

In [10]:
tuner = kt.BayesianOptimization(
    hypermodel=hypermodel,
    objective="val_binary_accuracy",
    max_trials=3,
    seed=seed
)

INFO:tensorflow:Reloading Oracle from existing project .\untitled_project\oracle.json
INFO:tensorflow:Reloading Tuner from .\untitled_project\tuner0.json


In [11]:
tuner.search(train_generator, validation_data=val_generator, steps_per_epoch=train_generator.n // batch_size, validation_steps=val_generator.n // batch_size)

Trial 3 Complete [00h 02m 29s]
val_binary_accuracy: 0.6754105687141418

Best val_binary_accuracy So Far: 0.9603102207183838
Total elapsed time: 00h 02m 29s
INFO:tensorflow:Oracle triggered exit
